# --- Day 15: Chiton --- 

https://adventofcode.com/2021/day/15

## Get Input Data

In [9]:
import numpy as np

In [61]:
def parse_data(filename):
    """Read in risk map data."""

    with open(f'../inputs/{filename}') as file:
        risk_map = [[int(x) for x in line.strip()] for line in file.readlines()]

    risk_map = np.matrix(risk_map)
    return risk_map

In [62]:
test_risk_map = parse_data('test_risk_map.txt')
test_risk_map

matrix([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
        [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
        [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
        [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
        [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
        [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
        [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
        [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
        [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
        [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [15]:
smaller_test_risk_map = test_risk_map[:3, :3]
smaller_test_risk_map

matrix([[1, 1, 6],
        [1, 3, 8],
        [2, 1, 3]])

In [71]:
# Get row slices
smaller_test_risk_map[:2, ...]

matrix([[1, 1, 6],
        [1, 3, 8]])

In [72]:
# Get column slices
smaller_test_risk_map[..., :2]

matrix([[1, 1],
        [1, 3],
        [2, 1]])

In [16]:
risk_map = parse_data('risk_map.txt')
risk_map

matrix([[6, 8, 9, ..., 4, 7, 4],
        [5, 3, 7, ..., 2, 9, 8],
        [8, 7, 9, ..., 8, 5, 1],
        ...,
        [9, 1, 2, ..., 8, 5, 6],
        [3, 7, 9, ..., 5, 8, 4],
        [9, 9, 8, ..., 8, 9, 2]])

In [29]:
risk_map.shape

(100, 100)

## Part 1
---

Will need to apply [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) to solve.

Chapter 7 in [Grokking Algorithms](https://www.manning.com/books/grokking-algorithms) provides an excellent tutorial on the algorithm -- highly recommend this book:

[![Grokking Algorithms](https://images.manning.com/360/480/resize/book/3/0b325da-eb26-4e50-8a2a-46042c647083/Bhargava-Algorithms_hires.png)](https://www.manning.com/books/grokking-algorithms)

Code for the algorithm can be found [here](https://github.com/egonSchiele/grokking_algorithms/blob/master/07_dijkstras_algorithm/python/01_dijkstras_algorithm.py).

In [80]:
def get_neighbors(position, map):
    """Return a list of [(y, x)] neighbor coordinates."""

    y, x = position[0], position[1]
    neighbors = [(y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    max_height, max_length = map.shape[0]-1, map.shape[1]-1

    # Filter out neighbors who have gone over the edge!
    good_neighbors = list(filter(lambda n: 0 <= n[0] <= max_height and 0 <= n[1] <= max_length, neighbors))

    return good_neighbors

In [90]:
def make_graph(map):
    """Convert a map into a dictionary/graph with appropriate weight/risk values for each node."""

    graph = {}

    for y in range(map.shape[0]):
        for x in range(map.shape[1]):
            neighbors = get_neighbors((y, x), map)

            graph[(y, x)] = {}
            for n in neighbors:
                graph[(y, x)][n] = map[n]

            print(f'({y}, {x}): neighbors: {neighbors}')

    # Set the "finish" node to have no neighbors
    graph[(map.shape[0]-1, map.shape[1]-1)] = {}

    return graph

In [91]:
make_graph(smaller_test_risk_map)

(0, 0): neighbors: [(1, 0), (0, 1)]
(0, 1): neighbors: [(1, 1), (0, 2), (0, 0)]
(0, 2): neighbors: [(1, 2), (0, 1)]
(1, 0): neighbors: [(2, 0), (0, 0), (1, 1)]
(1, 1): neighbors: [(2, 1), (0, 1), (1, 2), (1, 0)]
(1, 2): neighbors: [(2, 2), (0, 2), (1, 1)]
(2, 0): neighbors: [(1, 0), (2, 1)]
(2, 1): neighbors: [(1, 1), (2, 2), (2, 0)]
(2, 2): neighbors: [(1, 2), (2, 1)]


{(0, 0): {(1, 0): 1, (0, 1): 1},
 (0, 1): {(1, 1): 3, (0, 2): 6, (0, 0): 1},
 (0, 2): {(1, 2): 8, (0, 1): 1},
 (1, 0): {(2, 0): 2, (0, 0): 1, (1, 1): 3},
 (1, 1): {(2, 1): 1, (0, 1): 1, (1, 2): 8, (1, 0): 1},
 (1, 2): {(2, 2): 3, (0, 2): 6, (1, 1): 3},
 (2, 0): {(1, 0): 1, (2, 1): 1},
 (2, 1): {(1, 1): 3, (2, 2): 3, (2, 0): 2},
 (2, 2): {}}

In [74]:
foo, bar = smaller_test_risk_map.shape
print(foo, bar)

3 3


In [51]:
def find_minimum_risk_path(risk_graph):
    
    visited = [position]
    total_risk = risk_dict[position]

    while position != (0, 0):
        print(f'visited so far: {visited}')
        print(f'total risk so far: {total_risk}\n')
        position = make_move(position, start_position, risk_dict, visited)
        visited.append(position)
        total_risk += risk_dict[position]

    return total_risk

### Run on Test Data

In [ ]:
find_minimum_risk_path(smaller_test_risk_map)  # Should return 7

In [52]:
find_minimum_risk_path(test_risk_map)  # Should return 40

visited so far: [(9, 9)]
total risk so far: 1

current position: (9, 9)
good neighbors: [(8, 9), (9, 8)]
best neighbor: (8, 9), with a risk of 1
visited so far: [(9, 9), (8, 9)]
total risk so far: 2

current position: (8, 9)
good neighbors: [(7, 9), (8, 8)]
best neighbor: (8, 8), with a risk of 2
visited so far: [(9, 9), (8, 9), (8, 8)]
total risk so far: 4

current position: (8, 8)
good neighbors: [(8, 7), (9, 8), (7, 8)]
best neighbor: (7, 8), with a risk of 3
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8)]
total risk so far: 7

current position: (7, 8)
good neighbors: [(7, 9), (6, 8), (7, 7)]
best neighbor: (6, 8), with a risk of 2
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8), (6, 8)]
total risk so far: 9

current position: (6, 8)
good neighbors: [(6, 7), (6, 9), (5, 8)]
best neighbor: (6, 9), with a risk of 1
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8), (6, 8), (6, 9)]
total risk so far: 10

current position: (6, 9)
good neighbors: [(7, 9), (5, 9)]
best neighbor: (5, 9), wi

ValueError: min() arg is an empty sequence

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data